- To rerun any S-LOCAL network you need a minimum of 32GB of RAM/VRAM memory on the device used for training
- Average time to run a 100 epoch on a Nvidia Tesla V100 GPU is 20 minutes
- A faster implementation of the Locally Connected Layer is available in the Keras API : 
- https://keras.io/api/layers/locally_connected_layers/locall_connected2d/

## Imports

In [1]:
INIT_PATH = "."
DOWNLOAD_DATA = False

# To use google colab processing uncomment following lines

#INIT_PATH = "MyDrive/inference_and_learning"
#from google.colab import drive
#MOUNT_PATH = "/content/drive"
#drive.mount(MOUNT_PATH)
#INIT_PATH = MOUNT_PATH + "/" + INIT_PATH


# To train LOCAL change to True
LOCAL_TRAIN = False 

In [2]:
import os.path,sys
import networks as nnets
import pandas as pd, torch
from typing import Tuple, List
import torchvision
import torch
import json
torch.set_default_dtype(torch.float16)
sys.path.append(INIT_PATH)
torch.set_default_dtype(torch.float16)

In [3]:
DATA_PATH = "." # Path to data folder
if (DOWNLOAD_DATA):
    torchvision.datasets.CIFAR10(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.CIFAR100(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True)

    torchvision.datasets.CIFAR10(root=DATA_PATH +'/data', download=True,train = False)
    torchvision.datasets.CIFAR100(root=DATA_PATH +'/data', download=True ,train = False) 
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True, split = "test")


## OPTIONAL: optimization of parameters

In [7]:
# All network described in table 2 can be optimized with this script
optim = json.load(open(INIT_PATH+"/network_description/optim.json"))
optimer = nnets.NetworkOptim(optim["optim"],DATA_PATH,verbose = True,very_verbose = True)

for key,item in optim["networks"].items():
    if key in optim["results"]:
        print(f"Skipping {key} as it is already done")
        continue
    if not LOCAL_TRAIN and item["model"]["name"] in ["S-LOCAL","D-LOCAL"]:
        print(f"Skipping {key} as it is a localy connected network")
        continue
    print(f"Starting {item['model']['name']} with {item['optimizer']['name']} on {item['dataset']['name']}")
    acc,best = optimer(item)
    print(f"Best : {best} with accuracy {acc}")
    optim["results"][key] = {"best":best,"accuracy":acc}
    json.dump(optim,open(INIT_PATH+"/network_description/optim.json","w"))

Skipping S-FC_B-lasso(B=0)_CIFAR-10 as it is already done
Starting S-FC with B-lasso on CIFAR-10
Calculating accuracy for default parameters
batch_size : 512
epoch      : 100
use_cuda   : True
optimizer :
	name        : B-lasso
	lr          : 0.1
	B           : 50
	l1_coeff    : 1e-06
	l1_coeff_FC : 2e-05
dataset :
	name       : CIFAR-10
	transforms :
		0 : ToTensor
model :
	name      : S-FC
	alpha     : 160
	dropout_1 : 0
	dropout_2 : 0
scheduler :
	name       : CosineAnnealingWarmRestarts
	T_0        : 20
	T_mult     : 2
	eta_min    : 0
	last_epoch : -1
Training Network: S-FC, CIFAR-10, B-lasso using : 283.28833 M parameters on cuda
30.55%, 39.59%, 44.99%, 43.82%, 48.62%, 49.56%, 49.63%, 51.06%, 54.20%, 53.52%,  : 10/100
57.01%, 57.52%, 58.89%, 58.99%, 59.30%, 59.12%, 59.34%, 59.32%, 59.31%, 59.34%,  : 20/100
44.46%, 46.60%, 44.48%, 51.73%, 52.45%, 51.97%, 57.79%, 58.79%, 40.42%, 48.72%,  : 30/100
59.00%, 57.96%, 60.02%, 59.93%, 52.30%, 58.37%, 59.92%, 59.88%, 60.04%, 59.79%,  : 40/1

## Training

### table2

In [ ]:
table2 = pd.read_csv(INIT_PATH+"/csv/table2.csv")
table2_net = json.load(open(INIT_PATH+"/network_description/table2.json"))
RETRAIN = True

if RETRAIN :
    for dataset in ["CIFAR-10","CIFAR-100","SVHN"]:
        for l in range(len(table2)):
            network = table2.iloc[l].to_dict()
            if network["train"]=="False" or (network[dataset] == "NaN"):
                continue
            ID = "_".join([network[i] for i in ["Model","Training_Method"]]+[dataset])
            if not LOCAL_TRAIN and table2_net[ID]["model"]["name"] in ["S-LOCAL","D-LOCAL"]:
                print(f"Skipping {key} as it is a localy connected network")
                continue
            acc,_,_ = nnets.train(table2_net[ID],DATA_PATH,verbose = True) 
            table2.loc[l,dataset] = acc
            table2.to_csv(INIT_PATH+"/csv/table2.csv",index = False)

print(table2)

### figure 3

### figure 4